In [4]:
import pandas as pd
import numpy as np
import re
import os
import nlpaug.augmenter.word as naw
import nltk

# Uncomment the following lines on the first run to download necessary NLTK data
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

class Task2bPreprocessor:
    """
    Preprocessing pipeline for SemEval Task 2B (Disposition Change).

    Key Features:
    1. Handles User-Level data (streams of texts per user).
    2. Performs strict temporal sorting (User -> Group -> Time).
    3. Implements data augmentation suitable for regression tasks.
    """
    def __init__(self, augment=False):
        """
        Initialize the preprocessor.

        :param augment: Boolean. Whether to enable data augmentation.
                        (Set to True for Training Set, False for Validation/Test Sets).
        """
        self.augment = augment

        # Initialize augmenter: Synonym Replacement
        # This technique preserves the overall 'disposition' or personality tone
        # of the user while varying the vocabulary, which is ideal for this task.
        if self.augment:
            print(">>> [Init] Initializing Data Augmentation Model (WordNet)...")
            self.aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.2) # Replace 20% of words

    def clean_text(self, text, is_words_mode=False):
        """
        Text cleaning logic specific to Ecological Essays.
        Handles both full sentences and 'word list' formats.
        """
        if pd.isna(text): return ""
        text = str(text)

        # 1. Fix excessive ellipses (e.g., "I'm fine...........")
        # Reduce 2 or more dots to a single dot.
        text = re.sub(r'\.{2,}', '.', text)

        # 2. Handle 'Word List' format (is_words=True)
        # Converts "Happy , Sad , Calm" -> "Happy Sad Calm" for better tokenization.
        if is_words_mode:
            text = text.replace(' , ', ' ').replace(',', ' ')

        # 3. Normalize whitespace and expand contractions
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r"I ' m", "I am", text)
        text = re.sub(r"it ' s", "it is", text)
        text = re.sub(r"don ' t", "do not", text)

        return text

    def process(self, file_path, output_path):
        """
        Main processing function.
        Reads raw data, cleans it, sorts it, and optionally augments it.
        """
        print(f"--- Processing File: {file_path} ---")

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Input file not found: {file_path}")

        df = pd.read_csv(file_path)
        print(f"Original dataset shape: {df.shape}")

        # --- Step 1: Strict Temporal Sorting ---
        # Task 2B models (LSTM/Transformer) need to read a user's texts in chronological order.
        # We sort by: User ID -> Group (Time Period) -> Timestamp.
        if 'timestamp' in df.columns:
            print("Sorting data by User, Group, and Timestamp...")
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.sort_values(by=['user_id', 'group', 'timestamp'])

        # --- Step 2: Text Cleaning ---
        print("Cleaning text data...")
        # Check if 'is_words' column exists to apply specific cleaning
        if 'is_words' in df.columns:
            df['processed_text'] = df.apply(
                lambda row: self.clean_text(row['text'], row['is_words']), axis=1
            )
        else:
            # Fallback if 'is_words' column is missing
            df['processed_text'] = df['text'].apply(lambda x: self.clean_text(x))

        # --- Step 3: Data Augmentation (Training Set Only) ---
        if self.augment:
            print("Applying Data Augmentation (Generating 'Parallel User' samples)...")
            augmented_rows = []

            # Iterate through each row to generate augmented text
            for _, row in df.iterrows():
                original_text = row['processed_text']

                # Only augment if the sentence is long enough (> 3 words)
                if len(original_text.split()) > 3:
                    try:
                        # Generate augmented text (synonym replacement)
                        aug_text = self.aug.augment(original_text)[0]

                        new_row = row.copy()
                        new_row['processed_text'] = aug_text
                        new_row['is_augmented'] = True # Mark as augmented

                        augmented_rows.append(new_row)
                    except:
                        pass # Skip if augmentation fails

            # Merge augmented data
            aug_df = pd.DataFrame(augmented_rows)
            df['is_augmented'] = False # Mark original data

            # NOTE for Dataloader Design:
            # We append augmented data to the end.
            # When creating batches, 'augmented users' should be treated as
            # independent samples from 'original users'.
            df = pd.concat([df, aug_df], ignore_index=True)
            print(f"Augmentation complete. Added {len(aug_df)} new samples.")

        # Save to CSV
        df.to_csv(output_path, index=False)
        print(f"Processing finished! File saved to: {output_path}")
        return df

# ==========================================
# Main Execution Block
# ==========================================
if __name__ == "__main__":
    # 1. Define File Path
    # Logic: Look for 'data' folder in the parent directory
    # Expected structure: project_root/data/train_release_3sep2025/train_subtask2b_detailed.csv

    base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()

    # Construct path to the detailed CSV file
    input_file = os.path.join(base_dir, "..", "data", "train_release_3sep2025", "train_subtask2b_detailed.csv")

    # Fallback: Check current directory if the file is not found in the data folder
    if not os.path.exists(input_file):
        print(f"File not found in data folder. Checking current directory...")
        input_file = "train_subtask2b_detailed.csv"

    output_file = "train_subtask2b_processed.csv"

    # 2. Instantiate and Run
    # Set augment=True to generate the Training set
    processor = Task2bPreprocessor(augment=True)

    try:
        processor.process(input_file, output_file)

        # 3. Preview Results
        print("\n--- Data Preview (Head of User 1) ---")
        df_res = pd.read_csv(output_file)

        # Display key columns to verify sorting and cleaning
        cols = ['user_id', 'timestamp', 'processed_text', 'disposition_change_valence']
        if 'user_id' in df_res.columns:
            print(df_res[df_res['user_id'] == 1][cols].head())
        else:
            print(df_res.head())

    except Exception as e:
        print(f"\n[Error] An error occurred: {e}")

>>> [Init] Initializing Data Augmentation Model (WordNet)...
--- Processing File: /Users/van/Desktop/LLM/2025/Project/SemEval2026-EmoVA/src/../data/train_release_3sep2025/train_subtask2b_detailed.csv ---
Original dataset shape: (2764, 17)
Sorting data by User, Group, and Timestamp...
Cleaning text data...
Applying Data Augmentation (Generating 'Parallel User' samples)...


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package ave

Augmentation complete. Added 0 new samples.
Processing finished! File saved to: train_subtask2b_processed.csv

--- Data Preview (Head of User 1) ---
   user_id            timestamp  \
0        1  2021-06-09 12:41:57   
1        1  2021-06-11 12:01:45   
2        1  2021-06-13 13:15:07   
3        1  2021-06-16 12:03:12   
4        1  2021-06-17 12:38:38   

                                      processed_text  \
0  I feel good . I caught up on some sleep . Work...   
1  I’ve been feeling good for days and days . I r...   
2  I’ve been feeling fine personally . I’ve been ...   
3  I feel great . I’ve had a day off . I’m going ...   
4  I feel great today . I am well rested . I have...   

   disposition_change_valence  
0                    0.666667  
1                    0.666667  
2                    0.666667  
3                    0.666667  
4                    0.666667  


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package ave